In [1]:
import os
import sys 

path_folder = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/'
omitir = ''


def search_subfolders(path:str):
    '''Funcion para agregar rutas al path de ejecucion'''
    folder = []
    for root, dirs, _ in os.walk(path, topdown=False):
        for name in dirs:
            if name == omitir:
                print(f"[INFO] carpeta omitida: {name}")
            else:f
                folder.append(os.path.join(root, name))
    return folder


for i in search_subfolders(path_folder):
    sys.path.insert(0, i)

In [2]:
import pandas as pd
import json

In [3]:
from features_postgres import HandleDBpsql
from features_redis import HandleRedis
from class_load import LoadFiles

attempted relative import with no known parent package
Insercion de carpetas al path
attempted relative import with no known parent package
Insercion de carpetas al path


### Prueba de conexiones y libreria HandleDBpsql

In [6]:
LOGGER_CONFIG = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/logging.conf'
pg_handler = HandleDBpsql(logfile=LOGGER_CONFIG)

Rutas de archivos para crear tablas

Query para crear tabla en la base de datos

In [37]:
QUERY_DATA = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/querys/create_table.sql'
CONNECTION_PARAMETERS = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/database_postgrest.ini'

In [ ]:
parameter_connection = pg_handler.get_config_file(CONNECTION_PARAMETERS,section='postgresql')

query_data_ready = pg_handler.prepare_query(QUERY_DATA)

pg_handler.send_query(connection_parameters=CONNECTION_PARAMETERS, query=query_data_ready)

Rutas para insertar datos desde un archivo csv a postgres

Query para insertar los datos a postgres

In [17]:
QUERY_DATA = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/querys/insert_data.sql'
CONNECTION_PARAMETERS = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/database_postgrest.ini'
DATA_TO_INSERT = '/mnt/data extra/DATASET/PREDICTION_DATA/data_completa.csv'

In [19]:
query_data_ready = pg_handler.prepare_query(QUERY_DATA)


pg_handler.insert_data_from_csv(
    connection_parameters=CONNECTION_PARAMETERS,
    query=query_data_ready, 
    data_path=DATA_TO_INSERT)


INSERT INTO position (index, "Fecha Creación", "Material", "Material (Cod)", "Peso neto (TON)", "Solicitate", "Solicitate (Cod)", "Unidad Medida Base", "Unidad Medida Venta", "Cant Pedido UMV", "Valor Neto", "Contador Ped")             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)      


Rutas para Peticion de datos  a postgres

Query para Obtener los datos a postgres

In [9]:
QUERY_DATA = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/querys/get_table.sql'
CONNECTION_PARAMETERS = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/database_postgrest.ini'

In [10]:
query_data_ready = pg_handler.prepare_query(QUERY_DATA)

data_postgres = pg_handler.get_table(
    connection_parameters=CONNECTION_PARAMETERS,
    query=query_data_ready, 
    )

Rutas para Peticion de datos  a postgres

Query para Obtener los datos a postgres

In [4]:
QUERY_DATA = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/querys/get_table.sql'
CONNECTION_PARAMETERS = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/database_postgrest.ini'

In [16]:
columnas = ["Fecha Creación","Material (Cod)","Peso neto (TON)","Solicitate (Cod)","Cant Pedido UMV","Valor Neto"]
table = 'position'

data_replace = {
    'table':table,
    'columns':', '.join(['"' + columna + '"' for columna in columnas]),
    "order":"index",
    "where": "posicion > 1"
               }

query_data_ready = pg_handler.prepare_query_replace_value(QUERY_DATA,data_replace)

data_postgres = pg_handler.get_table(
    connection_parameters=CONNECTION_PARAMETERS,
    query=query_data_ready, 
    )

# pd.DataFrame(data_postgres)

### Prueba de conexiones y libreria HandleRedis

In [20]:
CONNECTION_PARAMETERS = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/database_postgrest.ini'
# INIT_DATA_REDIS ='/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/querys/template_redis.json'
LOGGER_CONFIG = '/home/bdebian/Documents/Projects/Stoke_prediccition/app_prediction/src/data/config/logging.conf'
DATA_TO_INSERT = '/mnt/data extra/DATASET/PREDICTION_DATA/data_completa.csv'

rd_handler = HandleRedis(logfile=LOGGER_CONFIG)
loader = LoadFiles()

data_to_redis = pd.read_csv(DATA_TO_INSERT,index_col=[0])

# create_data = loader.json_to_dict(INIT_DATA_REDIS)[0]

hasd_name = 'position'
create_data = {hasd_name:{}}

create_datahasd_name = {str(key): value for key, value in data_to_redis.to_dict('index').items()}

In [ ]:
#Escritura de informacion 
rd_handler.set_dict_data(hasd_name,create_data,CONNECTION_PARAMETERS )

In [23]:
#Lectura de informacion 
data = rd_handler.get_dict_data(hasd_name,CONNECTION_PARAMETERS )

In [30]:
#Conversion de informacion leida
pd.DataFrame(data['position'].values())

,Fecha Creación,Material,Material (Cod),Peso neto (TON),Solicitate,Solicitate (Cod),Unidad Medida Base,Unidad Medida Venta,Cant Pedido UMV,Valor Neto,Contador Ped
0,8/26/2022,Vinilo Basico Tipo 2 Galon Tito Pabon,96049073.0,0.0696,DEPOSITO SAN CARLOS,0004174201,UN,UN,12.0,297.0,1
1,3/3/2023,Sellotoc Gris 25kg Toxement,96005249.0,0.0500,LA VALENCIANA MATERIALES PARA,4171362.0,UN,UN,2.0,125.7,1
2,3/19/2022,Cemento Fuerte Ecoplanet 50 kg,10011308.0,35.0000,COSUM CASANARE,0004126992,TO,SAC,700.0,"10,059.3",1
3,1/4/2022,Holcim Oilwell/ Cem/Cl G/ Grado HSR/ Gra,10011318.0,34.0600,BASE BARRANCA,0004174934,TO,TO,34.1,"28,899.3",1
4,5/10/2022,CEMENTO INDS. FUERTE TIPO ART GRANEL,10070707.0,33.6800,HORMIGON ANDINO S A,0004172917,TO,TO,33.7,"11,308.1",1
...,...,...,...,...,...,...,...,...,...,...,...
105666,2/23/2022,Tanque Eternit 1.000 Lt C/Salida Negro,96003792.0,0.0285,MARTINEZ SUAREZ JOSE EIDER - CON -,0004135748,UN,UN,2.0,579.7,1
105667,6/16/2022,CEMENTO ESPECIAL PLUS (ESP+),10059053.0,33.5600,HOLCIM COLOMBIA KCHK,0004102519,TO,TO,33.6,"11,817.9",1
105668,1/29/2022,"Cemento Fuerte Ecoplanet 42,5kg",10011309.0,17.8500,"COSUM SAS - ARAUCA - 42,5 KGS -",0004154274,TO,SAC,420.0,"4,689.1",1
105669,4/6/2022,Cemento Fuerte Ecoplanet 25 kg,10060437.0,2.0000,DEPOSITO SAN CARLOS,0004174201,TO,SAC,160.0,"1,236.7",2
